In [3]:

#PATIENT SYMPTOM ANALYZER EASYGUI
from easygui import *
import csv
import time
import metapy
from datetime import datetime
from datetime import date
from time import strptime
from dateutil.relativedelta import relativedelta

#################### GET MRN FROM USER ###############################
msg = "Enter MRN:"
title = "Patient Lookup"
fieldNames = ["MRN"]
fieldValues = []  # we start with blanks for the values
fieldValues = multenterbox(msg,title, fieldNames)

# make sure that none of the fields was left blank
while 1:
    if fieldValues == None: break
    errmsg = ""
    for i in range(len(fieldNames)):
      if fieldValues[i].strip() == "":
        errmsg = errmsg + ('"%s" is a required field.\n\n' % fieldNames[i])
    if errmsg == "": break # no problems found
    fieldValues = multenterbox(errmsg, title, fieldNames, fieldValues)
    
#################### GET MRN FROM USER ###############################  



def tokenize_doc(doc):
    #tok = metapy.analyzers.ICUTokenizer(suppress_tags = True)
    tok = metapy.analyzers.ICUTokenizer()
    tok = metapy.analyzers.LowercaseFilter(tok)
    tok.set_content(doc_notes.content()) ##tok = tokenized sentence (each individual word in order)
    return(tok)

def negation_detection(word, num_words, tok):
    negation = False
    tokens = [token for token in tok]
    for i in range(len(tokens)):
        if num_words == 1:
            if tokens[i] == word:
                c = i-1;
                while (c >0 and i-c <= 15 and (tokens[c] != '<s>' and tokens[c] != ':' and tokens[c] != '-' and tokens[c] != ';')): 
                    for negation_word in negation_list:
                        if tokens[c] == negation_word['Negation']:
                            #print(tokens[c], i, ' ', c)
                            negation = True
                            return(negation)

                    c=c-1
                if ((i+4) < len(tokens)) and (tokens[i+1] == 'screening' or tokens[i+2] == 'screening' or tokens[i+3] == 'screening' or tokens[i+1] == 'exam' or tokens[i+2] == 'exam' or  tokens[i+3] == 'exam' or tokens[i+1] == 'education' or tokens[i+2] == 'education' or tokens[i+3] == 'education'):
                    negation = True
                    return(negation)
        if (num_words == 2 and (i+1) < len(tokens)):
            if (tokens[i] + ' ' + tokens[i+1]) == word:
                c = i-1;
                while (c >0 and i-c <= 15 and (tokens[c] != '<s>' and tokens[c] != ':' and tokens[c] != '-' and tokens[c] != ';')): 
                    for negation_word in negation_list:
                        if tokens[c] == negation_word['Negation']:
                            #print(tokens[c], i, ' ', c)
                            negation = True
                            return(negation)
                    c=c-1
                if ((i+5) < len(tokens)) and (tokens[i+2] == 'screening' or tokens[i+3] == 'screening' or tokens[i+4] == 'screening' or tokens[i+2] == 'exam' or tokens[i+3] == 'exam' or  tokens[i+4] == 'exam' or tokens[i+2] == 'education' or tokens[i+3] == 'education' or tokens[i+4] == 'education'):
                    negation = True
                    return(negation)
        if (num_words == 3 and (i+2) < len(tokens)):
            if (tokens[i] + ' ' + tokens[i+1] + ' ' + tokens[i+2]) == word:
                c = i-1;
                while (c >0 and i-c <= 15 and (tokens[c] != '<s>' and tokens[c] != ':' and tokens[c] != '-' and tokens[c] != ';')): 
                    for negation_word in negation_list:
                        if tokens[c] == negation_word['Negation']:
                            #print(tokens[c], i, ' ', c)
                            negation = True
                            return(negation)
                    c=c-1
                if ((i+6) < len(tokens)) and (tokens[i+3] == 'screening' or tokens[i+4] == 'screening' or tokens[i+5] == 'screening' or tokens[i+3] == 'exam' or tokens[i+4] == 'exam' or  tokens[i+5] == 'exam' or tokens[i+3] == 'education' or tokens[i+4] == 'education' or tokens[i+5] == 'education'):
                    negation = True
                    return(negation)
        if (num_words == 4 and (i+1) < len(tokens)):
            if (tokens[i] + ' ' + tokens[i+1] + ' ' + tokens[i+2] + ' ' + tokens[i+3]) == word:
                c = i-1;
                while (c >0 and i-c <= 15 and (tokens[c] != '<s>' and tokens[c] != ':' and tokens[c] != '-' and tokens[c] != ';')): 
                    for negation_word in negation_list:
                        if tokens[c] == negation_word['Negation']:
                            #print(tokens[c], i, ' ', c)
                            negation = True
                            return(negation)
                    c=c-1
                if ((i+7) < len(tokens)) and (tokens[i+4] == 'screening' or tokens[i+5] == 'screening' or tokens[i+6] == 'screening' or tokens[i+4] == 'exam' or tokens[i+5] == 'exam' or  tokens[i+6] == 'exam' or tokens[i+4] == 'education' or tokens[i+5] == 'education' or tokens[i+6] == 'education'):
                    negation = True
                    return(negation)
                
  
                
  

 #       if tokens[i] == word:
 #           if tokens[i-1] == 'denies':
 #               negation = True
 #               break            
    return(negation)

def build_rel_symptom_list(keys, num_words, doc):
    for key in keys:
        for symptom in symptom_list:
            if key == symptom['Symptom'].lower():
                #print(key)
                
                tok = tokenize_doc(doc)
                if negation_detection(keys[key], num_words, tok) == False:
                    rel_symptoms.append(key)
    return()

def unigram(doc):
    #keys = []
    keys = {}
    
    #word_part = []
    tok = tokenize_doc(doc) #tokenize document
    ana = metapy.analyzers.NGramWordAnalyzer(1,tok)  #stores results in tuples
    Ngrams = ana.analyze(doc_notes)
    for word in Ngrams:
        word_combination = word
        #keys.append(word_combination)
        keys[word] = word_combination
    build_rel_symptom_list(keys, 1, doc)
        
    return()    

def bigrams(doc):
    #keys = []
    keys = {}
    #word_part = []
    tok = tokenize_doc(doc) #tokenize document
    ana = metapy.analyzers.NGramWordAnalyzer(2,tok)  #stores results in tuples
    Ngrams = ana.analyze(doc_notes)
    for word in Ngrams:
        word_combination = word[0] + " " + word[1]
        #word_part.append(word[0])
        #keys.append(word_combination)
        keys[word_combination] = word_combination
    build_rel_symptom_list(keys, 2, doc)
        
 
    return()

def trigrams(doc):
    #keys = []
    keys = {}
    #word_part = []
    tok = tokenize_doc(doc) #tokenize document
    ana = metapy.analyzers.NGramWordAnalyzer(3,tok)  #stores results in tuples
    Ngrams = ana.analyze(doc_notes)
    for word in Ngrams:
        word_combination = word[0] + " " + word[1] + " " + word[2]
        #word_part.append(word[0])
        #keys.append(word_combination)
        keys[word_combination] = word_combination
    build_rel_symptom_list(keys, 3, doc)
 
            
    return()

def quadgrams(doc):
   #keys = []
    keys = {}
    #word_part = []
    tok = tokenize_doc(doc) #tokenize document
    ana = metapy.analyzers.NGramWordAnalyzer(4,tok)  #stores results in tuples
    Ngrams = ana.analyze(doc_notes)
    for word in Ngrams:
        word_combination = word[0] + " " + word[1] + " " + word[2] + " " + word[3]
        #word_part.append(word[0])
        #keys.append(word_combination)
        keys[word_combination] = word_combination
    build_rel_symptom_list(keys, 4, doc)
        

            
    return()


#open files symptoms.csv and store data in symptom_list
symptom_list = []
patient_list = []
negation_list = []
keys = [] #holds the keys that result in the NGramWordAnalyzer result
word_combination = "" #stores each tuple combination as phrase
rel_symptoms = []
with open('symptoms.csv','r') as csv_file:
    symptom_reader = csv.DictReader(csv_file)
    for row in symptom_reader:
        symptom_list.append(row)

#open files  patient_list.csv adn store data in patient_list
with open('patient_list.csv','r') as csv_file:
    patient_reader = csv.DictReader(csv_file)
    for row in patient_reader:
        patient_list.append(row)
        
#open files negation.csv and store data in negation_list
with open('negation.csv','r') as csv_file:
    negation_reader = csv.DictReader(csv_file)
    for row in negation_reader:
        negation_list.append(row)




#convert date from string to date

for patient in patient_list:
    DOS = patient['Date of Visit']
    DOB = patient['DOB']
    patient['Date of Visit'] = datetime.strptime(DOS, "%m/%d/%Y")
    patient['DOB'] = datetime.strptime(DOB, "%m/%d/%Y")
    
#pull patient of interest (eventually inputted by user) and pull DOS within last year: 
  #store data in patient_of_interest
if fieldValues != None:
    mrn = fieldValues[0]
    
patient_of_interest = []
for patient in patient_list:   
    if patient['Patient ID'] == mrn and (patient['Date of Visit'] > (datetime.now() - relativedelta(years=1))):
        patient_of_interest.append(patient)
#########Check if patient exists#############
patient_exists = []
for patient in patient_list:   
    if patient['Patient ID'] == mrn:
        patient_exists.append(patient)

        
        
        
# Loop through each encounter to generate a list of relevant symptoms     
for patient in patient_of_interest:
    doc_notes = metapy.index.Document()
    doc_notes.content(patient['Notes'])
    unigram(doc_notes)
    bigrams(doc_notes)
    trigrams(doc_notes)
    quadgrams(doc_notes)

#compile relevant symptoms
dict_symptoms = dict.fromkeys(rel_symptoms)
for symptom1 in rel_symptoms:
    symptom_count = 0
    for symptom in rel_symptoms:
        if symptom1 == symptom:
            symptom_count = symptom_count+1
        dict_symptoms[symptom1] = symptom_count

#sort symptoms
sorted_symptoms = sorted(dict_symptoms, key=lambda x: dict_symptoms[x], reverse=True)

f=open("results.txt","w")
   
#Print Patient Summary
if(len(patient_of_interest) == 0 and len(patient_exists) > 0):
    f.write("Patient was not seen in the last year.\n")
elif (len(patient_of_interest) == 1):    
    for patient in patient_of_interest:
        f.write("Patient summary:\n")
        f.write("Name: {}".format(patient['Patient Name']))
        f.write("\n")
        f.write("Gender: {}".format(patient['Gender']))
        f.write("\n")
        if (patient['DOB'].year > date.today().year):
            f.write("Age: {} years\n\n".format(date.today().year - (patient['DOB'].year - 100)))
        else:
            f.write("Age: {} years\n\n".format(date.today().year - (patient['DOB'].year)))
        break
    
    f.write("{} was seen once in the last year.\n\n".format(patient['Patient Name']))
    f.write("Key symptoms/conditions treated were:\n")
    for symptoms in sorted_symptoms:
        if (dict_symptoms[symptoms] == 1):
            f.write("-{} {} time.\n".format(symptoms, dict_symptoms[symptoms]))
        if (dict_symptoms[symptoms] > 1):
            f.write("-{} {} times.\n".format(symptoms, dict_symptoms[symptoms]))
    
    
else:
    for patient in patient_of_interest:
        f.write("Patient summary:\n")
        f.write("Name: {}".format(patient['Patient Name']))
        f.write("\n")
        f.write("Gender: {}".format(patient['Gender']))
        f.write("\n")
        if (patient['DOB'].year > date.today().year):
            f.write("Age: {} years\n\n".format(date.today().year - (patient['DOB'].year - 100)))
        else:
            f.write("Age: {} years\n\n".format(date.today().year - (patient['DOB'].year)))
        break

    f.write("{} was seen {} times in the last year.\n\n".format(patient['Patient Name'],len(patient_of_interest)))
    f.write("Key symptoms/conditions treated were:\n")
    #print("{} was seen {} times in the last year.".format(patient['Patient Name'],len(patient_of_interest)))
    #print("Key symptoms/conditions treated were:")
    for symptoms in sorted_symptoms:
        if (dict_symptoms[symptoms] == 1):
            f.write("-{} {} time.\n".format(symptoms, dict_symptoms[symptoms]))
        if (dict_symptoms[symptoms] > 1):
            f.write("-{} {} times.\n".format(symptoms, dict_symptoms[symptoms]))
            
f.close()

######## DISPLAY IN EASYGUI############
if fieldValues != None and len(patient_exists)>0:
    filename = "results.txt"
    f = open(filename, "r")
    text = f.readlines()
    f.close()
    codebox("", "", text)
######## DISPLAY IN EASYGUI############
if fieldValues == None:
    codebox("", "", "MRN was not entered.")
if len(patient_exists) == 0:
    codebox("", "", "MRN is not valid.")
    

